In [1]:
import os
import pandas as pd
import re
os.getcwd()



'c:\\Users\\399sh\\Desktop\\git\\Shivam_Learning\\Projects\\e2e_ecommerce_data_analysis-python'

Loading data from the sheet

In [4]:
data=pd.read_excel('Sample_Longlist_Data.xlsx')

In [5]:
data.head()

,Date Added,category,sellerlink,sellerlink-url,sellerstorefront-url,sellerproductcount,sellerratings,sellerdetails,seller business name,businessaddress,Count of seller brands,Max % of negative seller ratings - last 30 days,Max % of negative seller ratings - last 90 days,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,Sample brand name,Sample Brand URL
0,2020-11-15,Garden,Seller 1,Seller 1-link,Seller 1-storefrontlink,"1-16 of over 100,000 results",88% positive in the last 12 months (118 ratings),Lohas Living Inc James Mazzello US 845 3RD Ave...,Business Name:Lohas Living Inc,411 THEODORE FREMD AVESTE 206 SOUTHRyeNY10580-...,16,17,11,12,5873,4957,Visit the YAMAZAKI Store,https://www.amazon.de/-/en/stores/YAMAZAKI/pag...
1,2020-11-15,Garden,Seller 2,Seller 2-link,Seller 2-storefrontlink,NaN,90% positive in the last 12 months (566 ratings),Herzlich Willkommen im Amazon Shop von 1a-Hand...,Business Name:1a-Handelsagentur,1a HandelsagenturLindenallee 2MalchowMecklenbu...,0,2,2,9,0,0,Visit the tesa Store,https://www.amazon.de/-/en/stores/tesa/page/12...
2,2020-11-15,Garden,Seller 3,Seller 3-link,Seller 3-storefrontlink,"1-16 of over 2,000 results",85% positive in the last 12 months (75 ratings),67878u6i7 is committed to providing each custo...,Business Name:nantongaohuanfangzhipinyouxiango...,zhongnanshijicheng27zhuang1410haochongchuanqun...,15,0,25,12,164,162,TXYFYP,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...
3,2020-11-15,Garden,Seller 4,Seller 4-link,Seller 4-storefrontlink,1-16 of 123 results,NaN,7 days home life supermarket is committed to p...,Business Name:Zhengzhou Yaoqiong Trading Company,河南省中兴南路福禄街自贸试验区郑州片区（郑东）河南商会大厦1901室450000CN,1,0,0,0,0,0,SHIFENG,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...
4,2020-11-15,Garden,Seller 5,Seller 5-link,Seller 5-storefrontlink,"1-16 of over 1,000 results",81% positive in the last 12 months (52 ratings),78 68yui678 is committed to providing each cus...,Business Name:shenzhenfeichangshangmeimaoyiyou...,minzhidadaoshaxialaocun94dongA901longhuaxinqus...,16,0,31,15,224,113,YUOKI99,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...


seperating results from the seller product count

In [8]:
def parse_sellerproduct_count(data):
    try:
        sdata = data.split(' ')
        #print(sdata)
        temp = sdata[-2].replace(',','')
        #print(sdata[-2].replace(',',''))
        return int(temp)
    except:
        return data        

In [9]:
data["sellerproductcount_cleaned"] = data["sellerproductcount"].apply(parse_sellerproduct_count)
#print(data["sellerproductcount_cleaned"])

In [10]:
data.sellerratings.str.split(' ')[0]

['88%', 'positive', 'in', 'the', 'last', '12', 'months', '(118', 'ratings)']

separating percentage from the seller ratings

In [11]:
def parse_sellerratings_percent(data):
    try:
        sdata=data.split(' ')
        temp1=sdata[0].replace('%','')
        return int(temp1)
    except:
        return data        

In [12]:
data["sellerratings_percent_cleaned"] = data["sellerratings"].apply(parse_sellerratings_percent)

separating total ratings from the seller ratings

In [13]:
def parse_sellerratings_ratings(data):
    try:
        sdata=data.split(' ')
        temp1=sdata[-2].replace('(','')
        return int(temp1)
    except:
        try:
            sdata=data.split(' ')
            temp1=sdata[-3].replace('(','')
            return int(temp1)
        except:
            return data

In [14]:
data["sellerratings_ratings_cleaned"] = data["sellerratings"].apply(parse_sellerratings_ratings)

Extracting postive ratings from percentage and total ratings given by seller ratings

In [15]:
data["sellerratings_positive_ratings_cleaned"]=data["sellerratings_ratings_cleaned"]*data["sellerratings_percent_cleaned"]//100
data["sellerratings_positive_ratings_cleaned"]

0       103.0
1       509.0
2        63.0
3         NaN
4        42.0
        ...  
1834     38.0
1835      NaN
1836      0.0
1837      2.0
1838      2.0
Name: sellerratings_positive_ratings_cleaned, Length: 1839, dtype: float64

Extracting phone number from seller details

In [16]:
def parse_sellerdetails_phonenumbers(data):
    try:
        pattern=r'((Telefon|Telefax|Telefax|Fax|\+)+[:. ]?([0-9 -/().]*[0-9]))'
        sdata=data
        temp1=re.findall(pattern, sdata)
        temp1 = [str(a[0]).replace('Telefon:','') for a in temp1]
        return ','.join(temp1)
    except:
        return 'nan' 


In [17]:
data["sellerdetails_phonenumbers_cleaned"] = data["sellerdetails"].apply(parse_sellerdetails_phonenumbers)

In [18]:
data["sellerdetails_phonenumbers_cleaned"]

0       +1208-964-6807,+1208-964-6807
1                                    
2                                    
3                                    
4                                    
                    ...              
1834                                 
1835                                 
1836                                 
1837                                 
1838                                 
Name: sellerdetails_phonenumbers_cleaned, Length: 1839, dtype: object

Extracting mail id from seller details

In [19]:
def parse_sellerdetails_mailid(data):
    pattern='([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,})'#'^[a-z 0-9]+[\._]?[a-z 0-9]+[@]\w+[.]\w{2,3}$'
    try:
        sdata=data
        temp1=",".join(re.findall(pattern, sdata)) #.replace('E-Mail:','')  
        return temp1.replace('E-Mail:','')
    except:
        return data  

In [20]:
data["sellerdetails_mailid_cleaned"] = data["sellerdetails"].apply(parse_sellerdetails_mailid)

Extracting business address from seller details

In [21]:
def parse_businessaddress(data):
    try:
        sdata=data.replace(' ','')
        temp1=sdata[-2:]
        return temp1
    except:
        return 'nan' 

In [22]:
data["businessaddress_cleaned"] = data["businessaddress"].apply(parse_businessaddress)

Extracting business name from seller details

In [23]:
data["seller business name"][1].split(':')[-1]

'1a-Handelsagentur'

In [24]:
def seller_business_name(data):
    try:
        sdata=data.split(':')
        return sdata[-1]
    except:
        return 'nan'

In [25]:
data["seller_business_name"] = data["seller business name"].apply(seller_business_name)

In [26]:
data.head()

,Date Added,category,sellerlink,sellerlink-url,sellerstorefront-url,sellerproductcount,sellerratings,sellerdetails,seller business name,businessaddress,...,Sample brand name,Sample Brand URL,sellerproductcount_cleaned,sellerratings_percent_cleaned,sellerratings_ratings_cleaned,sellerratings_positive_ratings_cleaned,sellerdetails_phonenumbers_cleaned,sellerdetails_mailid_cleaned,businessaddress_cleaned,seller_business_name
0,2020-11-15,Garden,Seller 1,Seller 1-link,Seller 1-storefrontlink,"1-16 of over 100,000 results",88% positive in the last 12 months (118 ratings),Lohas Living Inc James Mazzello US 845 3RD Ave...,Business Name:Lohas Living Inc,411 THEODORE FREMD AVESTE 206 SOUTHRyeNY10580-...,...,Visit the YAMAZAKI Store,https://www.amazon.de/-/en/stores/YAMAZAKI/pag...,100000.0,88.0,118.0,103.0,"+1208-964-6807,+1208-964-6807",jadgemaello@gmail.com,US,Lohas Living Inc
1,2020-11-15,Garden,Seller 2,Seller 2-link,Seller 2-storefrontlink,NaN,90% positive in the last 12 months (566 ratings),Herzlich Willkommen im Amazon Shop von 1a-Hand...,Business Name:1a-Handelsagentur,1a HandelsagenturLindenallee 2MalchowMecklenbu...,...,Visit the tesa Store,https://www.amazon.de/-/en/stores/tesa/page/12...,NaN,90.0,566.0,509.0,,"info@1a-handelsagentur.de,info@1a-handelsagent...",DE,1a-Handelsagentur
2,2020-11-15,Garden,Seller 3,Seller 3-link,Seller 3-storefrontlink,"1-16 of over 2,000 results",85% positive in the last 12 months (75 ratings),67878u6i7 is committed to providing each custo...,Business Name:nantongaohuanfangzhipinyouxiango...,zhongnanshijicheng27zhuang1410haochongchuanqun...,...,TXYFYP,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...,2000.0,85.0,75.0,63.0,,,CN,nantongaohuanfangzhipinyouxiangongsi
3,2020-11-15,Garden,Seller 4,Seller 4-link,Seller 4-storefrontlink,1-16 of 123 results,NaN,7 days home life supermarket is committed to p...,Business Name:Zhengzhou Yaoqiong Trading Company,河南省中兴南路福禄街自贸试验区郑州片区（郑东）河南商会大厦1901室450000CN,...,SHIFENG,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...,123.0,NaN,NaN,NaN,,,CN,Zhengzhou Yaoqiong Trading Company
4,2020-11-15,Garden,Seller 5,Seller 5-link,Seller 5-storefrontlink,"1-16 of over 1,000 results",81% positive in the last 12 months (52 ratings),78 68yui678 is committed to providing each cus...,Business Name:shenzhenfeichangshangmeimaoyiyou...,minzhidadaoshaxialaocun94dongA901longhuaxinqus...,...,YUOKI99,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...,1000.0,81.0,52.0,42.0,,,CN,shenzhenfeichangshangmeimaoyiyouxiangongsi


In [27]:
len(data) # before filtering countries

1839

In [28]:
data=data[data["businessaddress_cleaned"]!='CN']
len(data) # after filtering countries

602

seperating cleaned data from the main data

In [29]:
Parsed_data=data[["sellerproductcount_cleaned","sellerratings_positive_ratings_cleaned","Count of seller brands","Max % of negative seller ratings - last 30 days","Max % of negative seller ratings - last 90 days","Max % of negative seller ratings - last 12 months","Hero Product 1 #ratings","Hero Product 2 #ratings","sellerdetails_phonenumbers_cleaned","sellerdetails_mailid_cleaned","businessaddress_cleaned","seller_business_name"]]
Parsed_data 

,sellerproductcount_cleaned,sellerratings_positive_ratings_cleaned,Count of seller brands,Max % of negative seller ratings - last 30 days,Max % of negative seller ratings - last 90 days,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,sellerdetails_phonenumbers_cleaned,sellerdetails_mailid_cleaned,businessaddress_cleaned,seller_business_name
0,100000.0,103.0,16,17,11,12,5873,4957,"+1208-964-6807,+1208-964-6807",jadgemaello@gmail.com,US,Lohas Living Inc
1,NaN,509.0,0,2,2,9,0,0,,"info@1a-handelsagentur.de,info@1a-handelsagent...",DE,1a-Handelsagentur
5,28.0,NaN,13,0,0,0,3284,2944,,,nan,nan
14,919.0,112.0,15,0,0,3,1446,921,015140008562,webmaster@9pm-store.de,DE,Oliver Mills
19,NaN,2827.0,0,18,20,20,0,0,Fax: 0570591156,abc-markt@web.de,DE,ABC-Schnäppchenmarkt GmbH
...,...,...,...,...,...,...,...,...,...,...,...,...
1808,52.0,16.0,8,0,0,0,65,41,,,DE,AD-ACCESS-Zündholz Riesa GmbH
1813,177.0,10.0,15,0,0,0,91,18,,"info@zurrosakuh.de,info@zurrosakuh.de",nan,DE815289413
1814,1000.0,2798.0,15,6,8,7,636,466,023626057826,"r.joormann@googlemail.com,r.joormann@googlemai...",DE,Reinhard Joormann -FansandTrends- e.K.
1815,2000.0,4.0,16,0,0,0,27269,11333,,,NL,Zwoofer.com


Normalising values

In [30]:
def Normalise(s):
    try:
        norm=(s-s.min())/(s.max()-s.min())
        return norm
    except:
        return data

In [40]:
Normalised_data=pd.DataFrame()
Normalised_data['Normalised_sellerproductcount'] = Normalise(Parsed_data["sellerproductcount_cleaned"])
Normalised_data['Normalised_sellerratings_positive_ratings'] = Normalise(Parsed_data["sellerratings_positive_ratings_cleaned"])
Normalised_data['Normalised_Count_of_seller_brands'] = Normalise(Parsed_data["Count of seller brands"])
Normalised_data['Normalised_negative_seller_ratings_last_12_months'] = Normalise(Parsed_data["Max % of negative seller ratings - last 12 months"])
Normalised_data['Hero_Product_1'] = Normalise(Parsed_data["Hero Product 1 #ratings"])
Normalised_data['Hero_Product_2'] = Normalise(Parsed_data["Hero Product 2 #ratings"])

Normalised_data['phonenumbers'] = Parsed_data['sellerdetails_phonenumbers_cleaned']
Normalised_data['mailid'] = Parsed_data['sellerdetails_mailid_cleaned']
Normalised_data['country'] = Parsed_data['businessaddress_cleaned']
Normalised_data['seller_business_name'] = Parsed_data['seller_business_name']

Normalised_data

,Normalised_sellerproductcount,Normalised_sellerratings_positive_ratings,Normalised_Count_of_seller_brands,Normalised_negative_seller_ratings_last_12_months,Hero_Product_1,Hero_Product_2,phonenumbers,mailid,country,seller_business_name
0,0.499992,0.002921,1.0000,0.12,0.067618,0.070081,"+1208-964-6807,+1208-964-6807",jadgemaello@gmail.com,US,Lohas Living Inc
1,NaN,0.014433,0.0000,0.09,0.000000,0.000000,,"info@1a-handelsagentur.de,info@1a-handelsagent...",DE,1a-Handelsagentur
5,0.000125,NaN,0.8125,0.00,0.037810,0.041622,,,nan,nan
14,0.004580,0.003176,0.9375,0.03,0.016648,0.013021,015140008562,webmaster@9pm-store.de,DE,Oliver Mills
19,NaN,0.080160,0.0000,0.20,0.000000,0.000000,Fax: 0570591156,abc-markt@web.de,DE,ABC-Schnäppchenmarkt GmbH
...,...,...,...,...,...,...,...,...,...,...
1808,0.000245,0.000454,0.5000,0.00,0.000748,0.000580,,,DE,AD-ACCESS-Zündholz Riesa GmbH
1813,0.000870,0.000284,0.9375,0.00,0.001048,0.000254,,"info@zurrosakuh.de,info@zurrosakuh.de",nan,DE815289413
1814,0.004985,0.079338,0.9375,0.07,0.007322,0.006588,023626057826,"r.joormann@googlemail.com,r.joormann@googlemai...",DE,Reinhard Joormann -FansandTrends- e.K.
1815,0.009985,0.000113,1.0000,0.00,0.313956,0.160225,,,NL,Zwoofer.com


Averaging both Hero product 1 and Hero product 2

In [41]:
Normalised_data['Hero_Product']=(Normalised_data['Hero_Product_1'] + Normalised_data['Hero_Product_2'])/2
Normalised_data

,Normalised_sellerproductcount,Normalised_sellerratings_positive_ratings,Normalised_Count_of_seller_brands,Normalised_negative_seller_ratings_last_12_months,Hero_Product_1,Hero_Product_2,phonenumbers,mailid,country,seller_business_name,Hero_Product
0,0.499992,0.002921,1.0000,0.12,0.067618,0.070081,"+1208-964-6807,+1208-964-6807",jadgemaello@gmail.com,US,Lohas Living Inc,0.068850
1,NaN,0.014433,0.0000,0.09,0.000000,0.000000,,"info@1a-handelsagentur.de,info@1a-handelsagent...",DE,1a-Handelsagentur,0.000000
5,0.000125,NaN,0.8125,0.00,0.037810,0.041622,,,nan,nan,0.039716
14,0.004580,0.003176,0.9375,0.03,0.016648,0.013021,015140008562,webmaster@9pm-store.de,DE,Oliver Mills,0.014835
19,NaN,0.080160,0.0000,0.20,0.000000,0.000000,Fax: 0570591156,abc-markt@web.de,DE,ABC-Schnäppchenmarkt GmbH,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
1808,0.000245,0.000454,0.5000,0.00,0.000748,0.000580,,,DE,AD-ACCESS-Zündholz Riesa GmbH,0.000664
1813,0.000870,0.000284,0.9375,0.00,0.001048,0.000254,,"info@zurrosakuh.de,info@zurrosakuh.de",nan,DE815289413,0.000651
1814,0.004985,0.079338,0.9375,0.07,0.007322,0.006588,023626057826,"r.joormann@googlemail.com,r.joormann@googlemai...",DE,Reinhard Joormann -FansandTrends- e.K.,0.006955
1815,0.009985,0.000113,1.0000,0.00,0.313956,0.160225,,,NL,Zwoofer.com,0.237090


Weightage for KPI's

1. Seller positive ratings(decending) - 40 weight
2. Max % of negative seller ratings - last 12 months (ascending) - 25 weight
3. Hero_Product - avg of Product 1 and 2(decending) - 15 weight
4. sellerproductcount(decending) - 10 weight
5. Count of seller brands (decending) - 10 weight

Total weight=100

In [54]:
def score(data):
    p1=data["Hero_Product"]
    p2=data['Normalised_sellerratings_positive_ratings']
    p3=data['Normalised_negative_seller_ratings_last_12_months']
    p4=data['Normalised_sellerproductcount']
    p5=data['Normalised_Count_of_seller_brands']
    return (15/100)*p1+ (40/100)*p2+(25/100)*(1-p3)+(10/100)*p4+(10/100)*p5

Normalised_data['score']=Normalised_data.apply(score,axis=1)
Normalised_data

,Normalised_sellerproductcount,Normalised_sellerratings_positive_ratings,Normalised_Count_of_seller_brands,Normalised_negative_seller_ratings_last_12_months,Hero_Product_1,Hero_Product_2,phonenumbers,mailid,country,seller_business_name,Hero_Product,score
495,0.499992,0.007429,0.8750,0.07,0.894745,1.000000,,,US,C. Y. MARKETING INC.,0.947373,0.515077
322,0.149987,0.018232,1.0000,0.06,0.825193,0.824690,"+49 59 71/ 8 00 38 03,Fax: +49 59 71/ 8 00 38 06",info@limuno.com,DE,Milovan Stojkovic,0.824942,0.481033
947,0.049986,0.004140,1.0000,0.06,1.000000,0.533224,,,JP,VIP Plaza Japan Inc,0.766612,0.456646
842,0.039986,0.001021,0.9375,0.03,0.825159,0.731706,,,IT,Vertecchi S.r.l.,0.778432,0.457422
321,0.034986,0.005217,1.0000,0.06,0.695220,0.816731,"0371 400 7682,Telefax: 0371 400 7685",shop@varia-store.com,DE,VARIA System GmbH,0.755975,0.453982
...,...,...,...,...,...,...,...,...,...,...,...,...
173,0.004985,0.000652,0.8750,0.08,0.178548,0.084856,,,GB,Baby Kooky Ltd,0.131702,0.338015
1051,0.024985,0.025349,0.8750,0.03,0.106406,0.039798,"+49 (0)36338 59889-00,Telefax: +49 (0)36338 5...",info@highlight-led.de,DE,World Trading Net GmbH & Co. KG,0.073102,0.353604
219,0.019985,0.006578,1.0000,0.01,0.024869,0.030481,"03741-221316,Telefax: 03741-289925",service@kurzwaren-profi.de,DE,Nähstüb`l Heike Bromnitz,0.027675,0.356281
629,0.004985,0.022202,1.0000,0.03,0.034644,0.029633,"+49 (0)5131- 50683 0,+49 (0)5131- 50683 30",kontakt@tierarzt24.de,DE,WDT Vision Trading GmbH & Co. KG,0.032138,0.356700


In [64]:
Normalised_data=Normalised_data.sort_values(by=['score'], ascending=False)
Normalised_data=Normalised_data[Normalised_data["score"] >= Normalised_data["score"].describe()["25%"]] 
#Normalised_data=Normalised_data[Normalised_data["score"] != ...]
Normalised_data

,Normalised_sellerproductcount,Normalised_sellerratings_positive_ratings,Normalised_Count_of_seller_brands,Normalised_negative_seller_ratings_last_12_months,Hero_Product_1,Hero_Product_2,phonenumbers,mailid,country,seller_business_name,Hero_Product,score
495,0.499992,0.007429,0.8750,0.07,0.894745,1.000000,,,US,C. Y. MARKETING INC.,0.947373,0.515077
882,0.019985,0.162191,0.9375,0.04,0.768813,0.458350,+4962369329325,"service@vitamed-quinger.com,Datenschutz@vitame...",DE,Vitamed Matthias Quinger e.K.,0.613581,0.492662
322,0.149987,0.018232,1.0000,0.06,0.825193,0.824690,"+49 59 71/ 8 00 38 03,Fax: +49 59 71/ 8 00 38 06",info@limuno.com,DE,Milovan Stojkovic,0.824942,0.481033
726,0.199988,0.182749,1.0000,0.09,0.468373,0.188189,"06206,Telefax: 06206",support@trendmile.com,DE,trendmile GmbH,0.328281,0.469840
100,0.034986,0.093941,1.0000,0.02,0.545006,0.458350,"+ 49 (0)2273 / 991-9325,Telefax: + 49 (0)2273...",info@kessler-electronic.de,DE,Kessler electronic GmbH,0.501678,0.461326
842,0.039986,0.001021,0.9375,0.03,0.825159,0.731706,,,IT,Vertecchi S.r.l.,0.778432,0.457422
36,0.249989,0.132872,0.9375,0.08,0.468396,0.267347,Fax: 06825 / 89697-20,info@itenga.de,DE,itenga GmbH,0.367872,0.457078
947,0.049986,0.004140,1.0000,0.06,1.000000,0.533224,,,JP,VIP Plaza Japan Inc,0.766612,0.456646
321,0.034986,0.005217,1.0000,0.06,0.695220,0.816731,"0371 400 7682,Telefax: 0371 400 7685",shop@varia-store.com,DE,VARIA System GmbH,0.755975,0.453982
1784,0.499992,0.073071,1.0000,0.02,0.297066,0.070392,,ade.support@zoreno.com,CH,Zoreno Versandhandel GmbH,0.183729,0.451787


Seperating Ranked data from the results

In [65]:
Ranked_data=Normalised_data[['seller_business_name','phonenumbers','mailid','score','country']]
Ranked_data

,seller_business_name,phonenumbers,mailid,score,country
495,C. Y. MARKETING INC.,,,0.515077,US
882,Vitamed Matthias Quinger e.K.,+4962369329325,"service@vitamed-quinger.com,Datenschutz@vitame...",0.492662,DE
322,Milovan Stojkovic,"+49 59 71/ 8 00 38 03,Fax: +49 59 71/ 8 00 38 06",info@limuno.com,0.481033,DE
726,trendmile GmbH,"06206,Telefax: 06206",support@trendmile.com,0.469840,DE
100,Kessler electronic GmbH,"+ 49 (0)2273 / 991-9325,Telefax: + 49 (0)2273...",info@kessler-electronic.de,0.461326,DE
842,Vertecchi S.r.l.,,,0.457422,IT
36,itenga GmbH,Fax: 06825 / 89697-20,info@itenga.de,0.457078,DE
947,VIP Plaza Japan Inc,,,0.456646,JP
321,VARIA System GmbH,"0371 400 7682,Telefax: 0371 400 7685",shop@varia-store.com,0.453982,DE
1784,Zoreno Versandhandel GmbH,,ade.support@zoreno.com,0.451787,CH
